In [ ]:
import subprocess
import yaml

from ska_dlm import CONFIG, data_item, dlm_ingest, dlm_migration, dlm_storage
from ska_dlm.dlm_db.db_access import DB

def _clear_database():
    DB.delete(CONFIG.DLM.dlm_table)
    DB.delete(CONFIG.DLM.storage_config_table)
    DB.delete(CONFIG.DLM.storage_table)
    DB.delete(CONFIG.DLM.location_table)

Set-up services:

In [ ]:
subprocess.run(['./setup-demo.sh'], shell=True)

In [ ]:
uid = dlm_ingest.register_data_item("/my/ingest/test/item", "/LICENSE", "MyDisk1")

In [15]:
# we need a location to register the storage:
location_id = dlm_storage.init_location(location_name="Pawsey", location_type="HPC centre", location_city="Perth", location_facility="Pawsey")
print(f"location_id of Pawsey Centre: {location_id}")

location_id of Pawsey Centre: 9270be09-2d98-4910-a5a6-f2b3d627e4c4


Define ACACIA as a storage endpoint:

In [16]:
uuid = dlm_storage.init_storage(
    storage_name="myacacia",
    location_id=location_id,
    storage_type="object store",
    storage_interface="s3",
    storage_capacity=5000000000,
    storage_phase_level="LIQUID",
)

Configure rclone:

In [17]:
with open(".secrets.yaml") as f:
    acacia_config = yaml.safe_load(f)["acacia_config"]
    
config_id = dlm_storage.create_storage_config(uuid, config=acacia_config)

Register a local data_item:

Register a data_item on ACACIA:

In [18]:
uid = dlm_ingest.register_data_item(item_name = "this/is/the/1st/package", uri = "rascil/1197634128-cal_avg32.ms.tar.xj", storage_name = "myacacia")

Copy the data_item from ACACIA to local:

In [19]:
copy_uid = dlm_migration.copy_data_item(item_name="this/is/the/1st/package", destination_name="MyDisk1", path="tmp/testdata")

'55471768-50af-45c1-967a-e03521ad90a4'

Query for all expired data_items:

In [21]:
data_item.set_uid_expiration('55471768-50af-45c1-967a-e03521ad90a4', "2024-01-01")

'55471768-50af-45c1-967a-e03521ad90a4'

In [22]:
!make python-post-test

scripts/teardown_services.sh .postgrest.pid .rclone.pid .dlmsm.pid
postgREST process killed successfully
rclone server process killed successfully
dlm sm service process killed successfully
[[ -z "$GITLAB_CI" ]] \
	&& make docker-compose-down \
	|| echo "Not stopping docker-compose containers in CI"
make[1]: Entering directory '/home/awicenec/git/ska-data-lifecycle'
docker compose --file docker/test-services.docker-compose.yml down
[+] Running 0/0
 ⠋ Container docker-ska-dlm-1  Stopping                                    0.1s 
[+] Running 0/1
 ⠙ Container docker-ska-dlm-1  Stopping                                    0.2s 
[+] Running 0/1
 ⠹ Container docker-ska-dlm-1  Stopping                                    0.3s 
[+] Running 0/1
 ⠸ Container docker-ska-dlm-1  Stopping                                    0.4s 
[+] Running 1/1
 ✔ Container docker-ska-dlm-1  Stopped                                     0.5s 
[+] Running 1/1
 ✔ Container docker-ska-dlm-1  Removed                         